In [2]:
PRED_PATH = "predictions_full.csv"  
EXPORT_PATH = "predictions_powerbi.csv"   

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# --- Load robustly: força leitura como texto para evitar problemas de separador decimal ---
df = pd.read_csv(PRED_PATH, dtype=str, low_memory=False)
print("Colunas lidas (tipos iniciais):")
print(df.dtypes)
print(df.head(3))


# ===========================================================
# 🔥 1. CONVERSÃO ROBUSTA PARA NÚMERO (REAL E PREVISTO)
# ===========================================================

target = 'UMIDADE_REL'
pred_col = target + '_pred'

def to_numeric_robust(series):
    s = series.astype(str).str.strip().str.replace(',', '.', regex=False)
    return pd.to_numeric(s, errors='coerce')

df[target] = to_numeric_robust(df[target])
df[pred_col] = to_numeric_robust(df[pred_col])

print("\nApós conversão numérica:")
print(df[[target, pred_col]].head())


# ===========================================================
# 🔥 2. CORRIGIR HORA “0900 UTC” → “09:00”
# ===========================================================

def fix_hour(h):
    if pd.isna(h):
        return None
    h = str(h).strip().upper().replace("UTC", "").strip()  # remove "UTC"
    if len(h) == 4 and h.isdigit():
        return f"{h[:2]}:{h[2:]}"  # ex.: "0900" → "09:00"
    return None

df["HORA_UTC_CLEAN"] = df["HORA_UTC"].apply(fix_hour)


# ===========================================================
# 🔥 3. Criar DATETIME (DATA + HORA) — opcional mas recomendado
# ===========================================================

# Converter DATA para datetime
df["DATA"] = pd.to_datetime(df["DATA"], dayfirst=True, errors="coerce")

# Se a hora vier vazia, usa 00:00
df["HORA_UTC_CLEAN"] = df["HORA_UTC_CLEAN"].fillna("00:00")

# Criar coluna DATETIME_UTC
df["DATETIME_UTC"] = pd.to_datetime(
    df["DATA"].dt.strftime("%Y-%m-%d") + " " + df["HORA_UTC_CLEAN"],
    errors="coerce"
)

print("\nExemplo de DATA + HORA convertidos:")
print(df[["DATA", "HORA_UTC", "HORA_UTC_CLEAN", "DATETIME_UTC"]].head())


# ===========================================================
# 🔥 4. MÉTRICAS
# ===========================================================

mae = mean_absolute_error(df[target].dropna(), df[pred_col].dropna())
mse = mean_squared_error(df[target].dropna(), df[pred_col].dropna())
rmse = np.sqrt(mse)

print(f"\nMAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")


# ===========================================================
# 🔥 5. ERRO ABSOLUTO + ARREDONDAMENTO
# ===========================================================

df["UMIDADE_ABS_ERROR"] = (df[target] - df[pred_col]).abs()

tiny_mask = df["UMIDADE_ABS_ERROR"].abs() < 1e-9
df.loc[tiny_mask, "UMIDADE_ABS_ERROR"] = 0.0

df[target] = df[target].round(3)
df[pred_col] = df[pred_col].round(3)
df["UMIDADE_ABS_ERROR"] = df["UMIDADE_ABS_ERROR"].round(3)


# ===========================================================
# 🔥 6. GRÁFICO
# ===========================================================

plt.figure(figsize=(12,4))
df[[target, pred_col]].iloc[:500].reset_index(drop=True).plot(title='Real vs Previsto (sample)')
plt.ylabel('Umidade (%)')
plt.tight_layout()
plt.savefig("plot_umidade.png")
plt.close()
print("Gráfico salvo como plot_umidade.png")


# ===========================================================
# 🔥 7. EXPORTAÇÃO LIMPA PARA O POWER BI
# ===========================================================

df.to_csv(EXPORT_PATH, index=False, float_format="%.2f")
print('Arquivo final para Power BI salvo em', EXPORT_PATH)


Colunas lidas (tipos iniciais):
TEMPERATURA_BULBO_SECO    object
PRESSAO_ATM_EST           object
RADIACAO_GLOBAL           object
UMIDADE_REL               object
ESTACAO                   object
DATA                      object
HORA_UTC                  object
UMIDADE_REL_pred          object
dtype: object
  TEMPERATURA_BULBO_SECO PRESSAO_ATM_EST RADIACAO_GLOBAL UMIDADE_REL  \
0                   20.6           937.2           130.1        85.0   
1                   22.3           937.7           841.0        78.0   
2                   25.0           938.2          1502.2        65.0   

      ESTACAO        DATA  HORA_UTC UMIDADE_REL_pred  
0  ARCO VERDE  2023/01/01  0900 UTC            84.41  
1  ARCO VERDE  2023/01/01  1000 UTC            79.39  
2  ARCO VERDE  2023/01/01  1100 UTC            67.03  

Após conversão numérica:
   UMIDADE_REL  UMIDADE_REL_pred
0         85.0             84.41
1         78.0             79.39
2         65.0             67.03
3         65.0         

<Figure size 1200x400 with 0 Axes>